In [2]:
import numpy as np
import pandas as pd
import cudf
import cupy
import dask_cudf
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
import rapids_qt
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import gc

cluster = LocalCUDACluster()
client = Client(cluster)

/home/aikawa/anaconda3/envs/rapids-24.10/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40207 instead
  warnings.warn(


In [3]:
V = cudf.read_csv('data/Liver_V.csv', header=0, index_col=0)
V.index.name = None

rank = 10
model = NMF(n_components=rank, init='nndsvd', random_state=42)
V = V.to_pandas()
W = model.fit_transform(V)
H = model.components_

/home/aikawa/anaconda3/envs/rapids-24.10/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
